In [1]:
# Get contends from an article and get each sentences
# Read sentences based on "，", not " " or "\n"
# Break sentences in words
# Then remove stop words
# Pool all this sentences in a list
# Save list in a csv file

import re
import jieba
import zhon


# Receive a list containing lines of an article
# Remove unwanted characters
# Retrun a list containing sentences of an article
def sentence_check(a):
    # The faster way is creating a list, then checking any unwanted characters before replacing it. 
    # However, this way produces multiple output, which increase some complexity 
    print("run sentence check")
    check_list=a
    return_list=[]
    pattern = r',|\.|/|;|\'|`|\[|\]|<|>|\?|:|"|\{|\}|\~|!|@|#|\$|%|\^|&|\(|\)|-|=|\_|\+|，|。|、|；|‘|’|【|】|·|！| |…|（|）' 
    
    for element in check_list:
        element = re.split(pattern,element)
        for t in element:
            if t != '':
                return_list.append(t)
    
    # Remove \n in list
    return_list = [x for x in return_list if x != '\n']
    return(return_list)


# Read sentences of an article and process them as a bag of words
# Use jieba module to recognize chinese words
# Need a dictionary of common tradition Chinese words
# Need a dictionary of stop words
# Return a list of lists of words (still keep the sentence information)
def word(a):
    print("Run word, powered by JieBa")
    input_list = a
    jieba.set_dictionary('dict.txt')
    stops = open('stop.txt', 'r', encoding='utf8') 
    stopword = stops.read().split('\n')
    stops.close()
    
    # Update stop words
    stopword.append('\n')
    
    new_list=[]
    i = 0
    
    # Remove any stop words from list
    while i < len(input_list):
        element = input_list[i]
        new_list.append([t for t in jieba.cut(element) if t not in stopword])
        i+=1
    
    return(new_list)
    
def main():
    print("Run main")
    fp = open('test.txt','r')
    lines = fp.readlines()
    fp.close()
    sentence_list = sentence_check(lines)
    word_list=word(sentence_list)
    print("Done!")
    
if __name__ == "__main__":
    print("initialize")
    main() 

Building prefix dict from /Users/hsieh/Documents/ptt/dict.txt ...
Loading model from cache /var/folders/54/w85qh3vd4xv5v6qz8w4t70940000gn/T/jieba.ubaa545e936adb4bd7ef290058a1bf422.cache


initialize
Run main
run sentence check
Run word, powered by JieBa


Loading model cost 0.752 seconds.
Prefix dict has been built succesfully.


Done!


In [2]:
import sqlite3
import requests
import csv
from bs4 import BeautifulSoup


# Enter the subforum which you want to scrape data from
# Subforum name is case sensitive

conn = sqlite3.connect('ptt.db')
c = conn.cursor()
cursor = c.execute("SELECT id from ARTICLE_INDEX")
url_list=[]
for row in cursor:
    url_list.append(row[0])
cursor.close()

name = input("Enter which sub forum you want to scrape data from: ")
print("get data from {a}".format(a = name))

# Send http request to server

text_list=[]
for element in url_list:
    article_id=re.findall("M.[0-9]{10}.A.[0-9a-zA-Z]{3}.html",element) 
    url = "https://www.ptt.cc/bbs/{name}/".format(name=name)
    url= url+article_id[0]
    response = requests.get(url, cookies={'over18': '1'}) 
    result = response.text
    soup = BeautifulSoup(result, 'html.parser')
    soup = soup.find(id="main-content")
    for span in soup('span'):
        span.decompose()
    for div in soup('div'):
        div.decompose()
    for a in soup('a'):
        a.decompose()
    text_list.append(soup.text)
print(text_list)
    

Enter which sub forum you want to scrape data from: Japan_Travel
get data from Japan_Travel
['\n\n\n\n沖繩跨年雙寶親子遊－分享2018-2019跨年廉航樂桃機票、OTS租車〈RA等級nissan之\nX-TRAIL休旅車〉費用～\n\n詳細圖文:\n\n\n\n雙十節假期臨時決定跨年沖一下～\n訂機票、訂房、租車動作都比別人慢，價錢較貴顯而易見，\n但想到大寶108/1/3滿二歲，終究還是決定107/12/30-108/1/2沖繩跨年！\n二打二太拼怕崩潰，所以親妹跟著我們一齊去，榮登最佳後援！\n\n由於要從高雄出發，傳統航空皆破萬至1.5萬，\n而虎航、香草時間不符小孩需求，最後選午去午回之樂桃，\n班機時間為：12/30、13:15-15:50；1/2、11:30-12:20，\n有來回選位第一排、來回行李20公斤，3大共31860元，2歲內之雙寶完全免費。\n\n高雄小港機場出發，大寶第一次出國去九州是6m對飛機沒印象，\n現在看到飛機超興奮一直說很大～\n會選位是因為想空間大點外，也想速速下飛機，\n本來預設雙寶會午睡，結果皆太雀躍，\n跟後座妹妹玩捉迷藏、吃零食，\n起降無法離位只能動用平板給大寶看巧虎大神，\n整個飛程超忙碌費心，ㄧ著地雙雙睡著！\n是根本來整我們的吧（白眼）\n\n樂桃停放沖繩LCC航廈（2019/3/18全面變更到國內線與國際線航廈之間的連結航廈3樓）\n，下飛機後可以看到整台飛機，\n結果我忘記儲存照片至相簿，入境卡地址只能機場寫，\n又加上廁所時間，整個先下飛機而最後入境(囧)\n\nots租車櫃檯在出境口右方，第十組，\n跟著人員坐接駁車至臨空豐崎營業所，需15分鐘，\n到達後以為跨年人潮多要排隊很久，結果約等五分鐘即到我們，\n辦理完畢後準備去領車囉！\n這在ots廁所內嬰兒尿布台，蠻乾淨，小寶身長快超過了～\n外頭有許多販賣機，買了冬天夯熱騰騰的玉米濃湯～\n\n恰巧ots有大感謝優惠活動，我們租12/30、16:00-1/1-19:00，\n櫃檯說我們在17:30還車可少租金，最後就是租48小時（12/30、17:30-1/1、17:30）加\netc卡、汽車座椅、保險費共20444日幣，\n這次跟ots租車主因是之前推